In [8]:
# ================================================================
#  Ground Station (GS) - GUI + LoRa bridge 
# ================================================================

# =========================
# [1] IMPORTS Y CONSTANTES
# =========================
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, filedialog
import serial
import serial.tools.list_ports
import threading
import time
import json
import pickle
from math import inf
import tkintermapview
from geographiclib.geodesic import Geodesic
import uuid

# Protocolo de comunicación
HEADER_OUT = "GS#"    # lo que ENVÍA la base
HEADER_IN  = "UAV#"   # lo que RECIBE la base
FOOTER     = "#END"

# Serial
SERIAL_PORT = None
SERIAL_BAUDRATE = 115200
ser = None
is_connected = False

# ACK
ACK_TIMEOUT = 1.5        # segundos para esperar ACK
MAX_RETRIES = 3          # reintentos máximos
pending_acks = {}        # { id: {"frame": str, "timestamp": float, "retries": int} }
ack_lock = threading.Lock()

# Estado
last_telemetry_pos = None

# Edición de mapa
EDIT_MODE = 'polygon'  # 'move', 'delete', 'home', 'polygon'

# =========================
# [2] ESTRUCTURAS DE DATOS
# =========================
class MissionData:
    def __init__(self):
        self.polygon_points = []   # [(lat, lon), ...]
        self.home_position = None  # (lat, lon) o None
        self.altitude = 30         # Altitud de misión (m)
        self.spacing = 10          # Ancho entre pasadas (m)
        self.takeoff_altitude = 10 # Altura de despegue (m)

    def to_dict(self):
        """Convierte a diccionario compacto para LoRa"""
        return {
            "p": [[round(lat, 6), round(lon, 6)] for lat, lon in self.polygon_points],
            "h": [round(self.home_position[0], 6), round(self.home_position[1], 6)] if self.home_position else None,
            "a": self.altitude,
            "s": self.spacing,
        }

    def from_dict(self, data):
        """Carga desde diccionario"""
        self.polygon_points = [tuple(point) for point in data.get("p", [])]
        self.home_position = tuple(data["h"]) if data.get("h") else None
        self.altitude = data.get("a", 30)
        self.spacing = data.get("s", 10)

# =========================
# [3] VARIABLES GLOBALES GUI
# =========================
# Mapa y marcadores
polygon_markers = []
polygon_path = None
home_marker = None
home_line = None
mission_path = None
waypoint_markers = []
waypoint_status = []
fire_markers = []
person_markers = []

# Controles / widgets
btn_disarm = None
btn_mission = None
btn_return = None
btn_arm = None
event_action_var = None
port_var = None
spacing_var = None
altitude_var = None
takeoff_altitude_var = None
status_message = None
home_position_var = None
progress_var = None
progress_bar = None

# Trim/Gripper
trim_accel_var = None
trim_roll_lr_var = None
trim_roll_fb_var = None
trim_rudder_var = None
trim_switch_var = None
slider_accel = None
slider_roll_lr = None
slider_roll_fb = None
slider_rudder = None
slider_switch = None
gripper_state_var = None
handle_trim_response_global = None  # se asigna en create_trim_controls()

# Vistas
root = None
mapa = None
telemetry_text = None
log_text = None

# =========================
# [4] UTILIDADES GEODÉSICAS
# =========================
def calculate_distance(lat1, lon1, lat2, lon2):
    """Distancia en metros entre dos puntos"""
    return Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['s12']

def calculate_bearing(lat1, lon1, lat2, lon2):
    """Rumbo en grados entre dos puntos"""
    return Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['azi1']

def get_point_at_distance(lat1, lon1, bearing, distance):
    """Punto desde (lat1, lon1) a distancia (m) y rumbo (°)"""
    result = Geodesic.WGS84.Direct(lat1, lon1, bearing, distance)
    return (result['lat2'], result['lon2'])

# =========================================================
# [5] LOG Y TELEMETRÍA (SEGUROS PARA LLAMAR DESDE HILOS)
# =========================================================
def log_message(message):
    """Agrega texto al log de manera segura desde cualquier hilo."""
    try:
        timestamp = time.strftime('%H:%M:%S')
        full_msg = f"{timestamp} - {message}\n"

        if 'log_text' not in globals() or not log_text:
            print(full_msg.strip())  # fallback si GUI aún no existe
            return

        def update_log():
            log_text.config(state="normal")
            log_text.insert("end", full_msg)
            log_text.see("end")
            log_text.config(state="disabled")

        root.after(0, update_log)
    except Exception as e:
        print(f"[Log Error] {e} :: {message}")

def update_telemetry(data):
    """Actualiza panel de telemetría y marcador del dron."""
    if not root:
        return

    def update_gui():
        global last_telemetry_pos
        telemetry_text.config(state="normal")
        telemetry_text.delete(1.0, "end")

        telemetry_str = f"Latitud: {data.get('lat', 'N/A')}\n"
        telemetry_str += f"Longitud: {data.get('lon', 'N/A')}\n"
        telemetry_str += f"Altitud: {data.get('alt', 'N/A')} m\n"
        telemetry_str += f"Velocidad: {data.get('speed', 'N/A')} km/h\n"
        telemetry_str += f"Rumbo: {data.get('heading', 'N/A')}°\n"
        telemetry_str += f"Batería: {data.get('battery', 'N/A')}\n"
        telemetry_str += f"Estado: {data.get('status', 'N/A')}\n"
        telemetry_str += f"Waypoint actual: {data.get('current_wp', 'N/A')}\n"
        telemetry_str += f"Waypoints totales: {data.get('total_wp', 'N/A')}\n"

        telemetry_text.insert("end", telemetry_str)
        telemetry_text.config(state="disabled")

        if 'lat' in data and 'lon' in data and data['lat'] != 'N/A' and data['lon'] != 'N/A':
            last_telemetry_pos = (data['lat'], data['lon'])
            update_drone_position(data['lat'], data['lon'])

    root.after(0, update_gui)

def update_drone_position(lat, lon):
    """Dibuja/actualiza el marcador del dron en el mapa."""
    if hasattr(update_drone_position, "marker") and update_drone_position.marker:
        update_drone_position.marker.delete()

    update_drone_position.marker = mapa.set_marker(
        lat, lon, text="DRON",
        marker_color_circle="red", marker_color_outside="red",
        text_color="red", font=("Helvetica", 12, "bold")
    )

    current_center = mapa.get_position()
    if calculate_distance(current_center[0], current_center[1], lat, lon) > 5000:
        mapa.set_position(lat, lon)
        mapa.set_zoom(15)

# =====================================
# [6] COMUNICACIÓN: CONEXIÓN / ENVÍOS
# =====================================
def connect_serial():
    """Conecta el puerto serial y lanza los hilos de recepción y ACK."""
    global ser, is_connected, SERIAL_PORT

    is_connected = False
    SERIAL_PORT = port_var.get()

    if not SERIAL_PORT:
        messagebox.showerror("Error", "Selecciona un puerto serial primero")
        return False

    try:
        ser = serial.Serial(SERIAL_PORT, SERIAL_BAUDRATE, timeout=1)
        ser.reset_input_buffer()
        ser.reset_output_buffer()

        is_connected = True
        print(f"[SERIAL] ✅ Conectado a {SERIAL_PORT} @ {SERIAL_BAUDRATE} bps")
        status_label.config(text=f"Conectado: {SERIAL_PORT}", fg="green")
        log_message(f"✅ Conectado correctamente a {SERIAL_PORT} a {SERIAL_BAUDRATE} bps")

        if 'root' in globals() and root:
            print("[SERIAL] 🚀 Lanzando hilos: recepción y ack_manager")
            threading.Thread(target=receive_data, daemon=True).start()
            threading.Thread(target=ack_manager, daemon=True).start()
        else:
            print("[SERIAL] ⚠️ GUI no inicializada aún, no se lanzan hilos")

        return True

    except Exception as e:
        is_connected = False
        print(f"[SERIAL] ❌ Error conectando a {SERIAL_PORT}: {e}")
        log_message(f"❌ Error conectando al puerto {SERIAL_PORT}: {e}")
        messagebox.showerror("Error", f"No se pudo conectar: {str(e)}")
        return False

def disconnect_serial():
    """Desconecta el puerto serial de forma segura y limpia."""
    global ser, is_connected

    if not is_connected:
        print("[SERIAL] ⚠️ No hay conexión activa para cerrar.")
        log_message("⚠️ No hay conexión activa para cerrar.")
        return

    try:
        is_connected = False
        time.sleep(0.2)  # permitir que terminen hilos
        if ser and ser.is_open:
            ser.close()
        status_label.config(text="Desconectado", fg="red")
        log_message("🔌 Puerto serial cerrado correctamente")
        print("[SERIAL] ✅ Puerto cerrado correctamente")
    except Exception as e:
        log_message(f"⚠️ Error al cerrar el puerto serial: {e}")
        print(f"[SERIAL] ⚠️ Error al cerrar puerto: {e}")
    finally:
        ser = None

def send_command(command_type, data=None, require_ack=True):
    """Forma el frame GS#...#END, envía y registra para ACK si corresponde."""
    if not is_connected:
        messagebox.showwarning("Error", "No hay conexión serial")
        return

    # ID único del paquete
    packet_id = str(uuid.uuid4())[:8]

    message = {
        "t": command_type,
        "d": data if data else {},
        "ts": int(time.time() * 1000),
        "id": packet_id
    }

    payload = json.dumps(message, separators=(",", ":"))
    frame   = f"{HEADER_OUT}{payload}{FOOTER}"

    try:
        # 🟦 Print de envío completo
        print(f"[TX → LoRa/send_command] Frame enviado → {frame}")
        
        ser.write(frame.encode())
        log_message(f"Enviado: {frame}")

        if require_ack and command_type != "TELEMETRY":
            with ack_lock:
                pending_acks[packet_id] = {
                    "frame": frame,
                    "timestamp": time.time(),
                    "retries": 0
                }

    except Exception as e:
        log_message(f"Error enviando: {str(e)}")

# ========================================
# [7] COMUNICACIÓN: RECEPCIÓN Y ACKs
# ========================================
def ack_manager():
    """Reintenta envíos sin ACK dentro del timeout."""
    while is_connected:
        time.sleep(0.2)
        now = time.time()

        with ack_lock:
            to_delete = []
            for pid, info in list(pending_acks.items()):
                if now - info["timestamp"] > ACK_TIMEOUT:
                    if info["retries"] < MAX_RETRIES:
                        try:
                            ser.write(info["frame"].encode())
                            pending_acks[pid]["timestamp"] = now
                            pending_acks[pid]["retries"] += 1
                            print(f"[ACK MANAGER → LoRa] 🔄 Reintentando ID={pid} (reintento {info['retries']})")
                            log_message(f"🔄 Reintentando envío ID={pid}")
                        except Exception as e:
                            print(f"[ACK MANAGER] ⚠️ Error reintentando ID={pid}: {e}")
                            log_message(f"Error reintentando ID={pid}: {e}")
                    else:
                        log_message(f"❌ No se recibió ACK para ID={pid}")
                        print(f"[ACK MANAGER] ❌ ACK no recibido tras {MAX_RETRIES} intentos → ID={pid}")
                        to_delete.append(pid)

            for pid in to_delete:
                del pending_acks[pid]

def receive_data():
    """Lee y procesa frames UAV#...#END desde el serial."""
    global is_connected
    buffer = ""

    while is_connected:
        if not ser or not ser.is_open:
            break
        try:
            if ser.in_waiting > 0:
                chunk = ser.read(ser.in_waiting).decode(errors="ignore")
                buffer += chunk

                while True:
                    h = buffer.find(HEADER_IN)
                    if h < 0:
                        if len(buffer) > 4096:
                            buffer = buffer[-512:]
                        break

                    f = buffer.find(FOOTER, h)
                    if f < 0:
                        if h > 0: buffer = buffer[h:]
                        break

                    payload = buffer[h + len(HEADER_IN):f].strip()
                    buffer = buffer[f + len(FOOTER):]

                    # 🟩 Print de recepción de paquete completo
                    print(f"[RX ← LoRa/receive_data] Frame recibido ← UAV#{payload}#END")

                    try:
                        msg = json.loads(payload)
                        process_received_message(msg)
                    except Exception as e:
                        log_message(f"❌ Mensaje inválido: {payload[:80]} ({e})")

            time.sleep(0.01)
        except Exception as e:
            log_message(f"Error recibiendo datos: {e}")
            time.sleep(0.1)

    log_message("📴 Hilo de recepción terminado")

def process_received_message(message):
    """
    Procesa cualquier JSON recibido desde el dron (UAV).
    Envía ACK automáticos cuando corresponde (manteniendo framing GS#/UAV# y #END).
    """
    msg_type = message.get("t", "")
    data = message.get("d", {})
    packet_id = message.get("id", data.get("id", None))

    print(f"[RX ← LoRa/process_received_message] Mensaje recibido tipo '{msg_type}' → {message}")

    if not msg_type:
        log_message(f"⚠️ Mensaje sin tipo: {message}")
        return

    # ACK entrante desde UAV
    if msg_type == "ACK":
        print(f"[ACK IN ← LoRa/process_received_message] {message}")
        ack_id = None
        if "id" in message:
            ack_id = message["id"]
        elif "d" in message and isinstance(message["d"], dict) and "id" in message["d"]:
            ack_id = message["d"]["id"]

        if ack_id:
            with ack_lock:
                if ack_id in pending_acks:
                    del pending_acks[ack_id]
                    log_message(f"✅ ACK recibido para ID={ack_id}")
                else:
                    log_message(f"⚠️ ACK recibido con ID desconocido: {ack_id}")
        else:
            log_message("⚠️ ACK recibido sin ID válido")
        return

    # Procesamiento de mensajes regulares
    if msg_type == "TELEMETRY":
        update_telemetry(data)
        #print(f"[TELEMETRY ← LoRa] Data: {data}")
        # Telemetría no requiere ACK por diseño
        return

    elif msg_type == "STATUS":
        print(f"[STATUS ← LoRa] Data: {data}")
        log_message(f"Estado: {data.get('message', '')}")
        status_message.set(f"Estado: {data.get('message', '')}")

    elif msg_type == "WP_REACHED":
        print(f"[WAYPOINT ← LoRa] Data: {data}")
        waypoint_reached(data.get("wp_number", 0))

    elif msg_type == "DEBUG":
        print(f"[DEBUG ← LoRa] Data: {data}")
        log_message(f"🪲 DEBUG: {data.get('message', '')}")

    elif msg_type == "FIRE":
        print(f"[FIRE ← LoRa] Data: {data}")
        add_fire_marker(data)

    elif msg_type == "PERSON":
        print(f"[PERSON ← LoRa] Data: {data}")
        add_person_marker(data)

    elif msg_type == "TRIM_DATA":
        # habilita sliders y carga valores
        print(f"[TRIM_DATA ← LoRa] Data: {data}")
        if handle_trim_response_global:
            handle_trim_response_global(data)

    else:
        print(f"[UNKNOWN ← LoRa] {message}")
        log_message(f"❔ Mensaje desconocido: {msg_type}")

    # Enviar ACK de respuesta si vino con ID
    if packet_id:
        ack_payload = {"id": packet_id}
        frame = f"{HEADER_OUT}" + json.dumps({"t": "ACK", "d": ack_payload, "id": str(uuid.uuid4())[:8]}) + f"{FOOTER}"
        try:
            ser.write(frame.encode())
            print(f"[ACK OUT → LoRa/process_received_message] Frame enviado → {frame}")
            log_message(f"↩️ Enviado ACK para ID={packet_id}")
        except Exception as e:
            log_message(f"⚠️ Error enviando ACK ID={packet_id}: {e}")

# ======================================
# [8] GESTIÓN DE MISIÓN (ENVÍOS, ESTADO)
# ======================================
def update_mission_parameters():
    current_mission.altitude = altitude_var.get()
    current_mission.spacing = spacing_var.get()
    current_mission.takeoff_altitude = takeoff_altitude_var.get()
    log_message("Parámetros de misión actualizados")

def send_compact_mission():
    if len(current_mission.polygon_points) < 3:
        messagebox.showwarning("Error", "Definí al menos 3 puntos en el mapa")
        return

    if not current_mission.home_position:
        messagebox.showwarning("Error", "Establece una posición HOME primero")
        return

    mission_dict = current_mission.to_dict()
    mission_dict["event_action"] = event_action_var.get()
    send_command("MISSION_COMPACT", mission_dict)
    print(f"[TX → LoRa/send_compact_mission] Datos misión compacta → {mission_dict}")
    log_message(f"Misión enviada (formato compacto) con acción tras evento: {event_action_var.get()}")

def start_mission():
    btn_return.config(state="normal", bg="orange")
    send_compact_mission()

def return_home():
    btn_disarm.config(state="normal", bg="red")
    btn_mission.config(state="disabled", bg="lightgrey")
    btn_return.config(state="disabled", bg="lightgrey")
    send_command("RETURN")
    print("[TX → LoRa/return_home] Enviando comando RETURN")

def arm_drone():
    if not last_telemetry_pos:
        log_message("⚠️ No hay datos de telemetría para fijar HOME")
        return
    send_command("ARM")
    log_message("🚁 Drone ARMADO")
    print("[TX → LoRa/arm_drone] Enviando comando ARM")
    btn_disarm.config(state="disabled", bg="lightgrey")
    btn_mission.config(state="normal", bg="green", fg="white")

def disarm_drone():
    global home_marker, home_line
    if home_marker:
        home_marker.delete(); 
    if home_line:
        home_line.delete()
    home_marker = None
    home_line = None
    current_mission.home_position = None
    home_position_var.set("")
    send_command("DISARM")
    print("[TX → LoRa/disarm_drone] Enviando comando DISARM")
    log_message("🛑 Drone DESARMADO")
    btn_mission.config(state="disabled", bg="lightgray", fg="black")
    btn_return.config(state="disabled", bg="lightgrey", fg="black")

# Stubs para evitar errores si se usan en el futuro
def at_home(): return True
def not_mission_started(): return True

def initialize_waypoint_tracking(total_waypoints):
    global waypoint_status
    waypoint_status = ["pending"] * total_waypoints
    update_waypoint_display()

def waypoint_reached(wp_number):
    if 0 <= wp_number < len(waypoint_status):
        waypoint_status[wp_number] = "completed"
        update_waypoint_display()
        log_message(f"Waypoint {wp_number} completado")

def update_waypoint_display():
    if waypoint_status:
        completed = waypoint_status.count("completed")
        total = len(waypoint_status)
        progress_var.set(f"Waypoints: {completed}/{total}")
        progress_bar['value'] = (completed / total) * 100

# ===========================================
# [9] MAPA: EDICIÓN, DIBUJO Y MARCADORES
# ===========================================
def set_edit_mode(mode):
    global EDIT_MODE
    EDIT_MODE = mode
    modes = {
        'polygon': "Añadir Puntos de Polígono",
        'move': "Mover Marcadores",
        'delete': "Eliminar Marcadores"
    }
    status_message.set(f"Modo: {modes.get(mode, mode)}")
    log_message(f"Modo cambiado a: {modes.get(mode, mode)}")
    update_button_colors()

def update_button_colors():
    for btn in [add_btn, delete_btn]:
        btn.config(bg="SystemButtonFace")
    if EDIT_MODE == 'polygon':
        add_btn.config(bg="lightblue")
    elif EDIT_MODE == 'delete':
        delete_btn.config(bg="lightcoral")

def map_click(coords):
    lat, lon = coords
    if EDIT_MODE == 'polygon':
        current_mission.polygon_points.append((lat, lon))
        marker = mapa.set_marker(lat, lon, text=f"P{len(current_mission.polygon_points)}")
        polygon_markers.append(marker)
        update_polygon_drawing()
        log_message(f"Punto de polígono añadido: {lat:.6f}, {lon:.6f}")
    elif EDIT_MODE == 'delete':
        closest_marker = find_closest_marker(lat, lon)
        if closest_marker:
            delete_marker(closest_marker)
        else:
            log_message("No se encontró ningún marcador cercano")

def find_closest_marker(lat, lon, max_distance=1000):
    closest_marker = None
    min_distance = inf
    for i, marker in enumerate(polygon_markers):
        marker_pos = marker.position
        dist = calculate_distance(lat, lon, marker_pos[0], marker_pos[1])
        if dist < min_distance and dist < max_distance:
            min_distance = dist
            closest_marker = ('polygon', i, marker)
    if home_marker:
        marker_pos = home_marker.position
        dist = calculate_distance(lat, lon, marker_pos[0], marker_pos[1])
        if dist < min_distance and dist < max_distance:
            min_distance = dist
            closest_marker = ('home', 0, home_marker)
    return closest_marker

def delete_marker(marker_info):
    global home_marker, home_line
    btn_mission.config(state="disabled", bg="lightgray", fg="black")
    marker_type, index, marker = marker_info
    if marker_type == 'polygon':
        current_mission.polygon_points.pop(index)
        marker.delete()
        polygon_markers.pop(index)
        for i, marker in enumerate(polygon_markers):
            marker.set_text(f"P{i + 1}")
        update_polygon_drawing()
        log_message(f"Punto P{index + 1} eliminado")
    elif marker_type == 'home':
        if home_marker:
            home_marker.delete(); 
        if home_line:
            home_line.delete()
        home_marker = None
        home_line = None
        current_mission.home_position = None
        home_position_var.set("")
        log_message("🏠 HOME eliminado")

def set_home_position(lat, lon):
    global home_marker, home_line
    if home_marker: home_marker.delete()
    if home_line: home_line.delete()
    home_marker = None
    home_line = None

    current_mission.home_position = (lat, lon)
    home_marker = mapa.set_marker(lat, lon, text="HOME",
                                  marker_color_circle="green", marker_color_outside="green")
    home_position_var.set(f"{lat:.6f}, {lon:.6f}")
    log_message(f"🏠 Home position establecida: {lat:.6f}, {lon:.6f}")

    if current_mission.polygon_points:
        closest_point = find_closest_polygon_point(lat, lon)
        home_line = mapa.set_path([current_mission.home_position, closest_point], color="green", width=2)

def set_home_from_telemetry():
    """Establece HOME desde última telemetría."""
    global last_telemetry_pos
    if not last_telemetry_pos:
        log_message("⚠️ No hay datos de telemetría para establecer HOME")
        messagebox.showwarning("Atención", "No hay datos de telemetría disponibles")
        return
    lat, lon = last_telemetry_pos
    set_home_position(lat, lon)
    log_message(f"🏠 HOME actualizado desde telemetría: {lat:.6f}, {lon:.6f}")

def find_closest_polygon_point(lat, lon):
    if not current_mission.polygon_points:
        return None
    min_distance = inf
    closest_point = None
    for point in current_mission.polygon_points:
        dist = calculate_distance(lat, lon, point[0], point[1])
        if dist < min_distance:
            min_distance = dist
            closest_point = point
    return closest_point

def add_fire_marker(data):
    lat = data.get('lat'); 
    lon = data.get('lon')
    print(f"[RX ← LoRa/add_fire_marker] Detección FIRE en ({lat}, {lon})")
    if lat is None or lon is None: return
    marker = mapa.set_marker(lat, lon, text="FUEGO",
                             marker_color_circle="orange", marker_color_outside="orange")
    fire_markers.append(marker)
    coord_marker = mapa.set_marker(lat + 0.0001, lon, text=f"{lat:.6f}, {lon:.6f}",
                                   marker_color_circle="white", marker_color_outside="white",
                                   text_color="black", font=("Arial", 9, "bold"))
    fire_markers.append(coord_marker)
    log_message(f"🔥 Marcador FUEGO agregado: {lat:.6f}, {lon:.6f}")

def add_person_marker(data):
    lat = data.get('lat'); 
    lon = data.get('lon')
    print(f"[RX ← LoRa/add_person_marker] Detección PERSON en ({lat}, {lon})")
    if lat is None or lon is None: return
    marker = mapa.set_marker(lat, lon, text="PERSONA",
                             marker_color_circle="#8B4513", marker_color_outside="#8B4513")
    person_markers.append(marker)
    coord_marker = mapa.set_marker(lat + 0.0001, lon, text=f"{lat:.6f}, {lon:.6f}",
                                   marker_color_circle="white", marker_color_outside="white",
                                   text_color="black", font=("Arial", 9, "bold"))
    person_markers.append(coord_marker)
    log_message(f"🧍 Marcador PERSONA agregado: {lat:.6f}, {lon:.6f}")

def update_polygon_drawing():
    global polygon_path
    if polygon_path:
        polygon_path.delete()
    if len(current_mission.polygon_points) >= 2:
        polygon_path = mapa.set_polygon(
            current_mission.polygon_points,
            outline_color="blue", fill_color="lightblue", border_width=2
        )

def reset_polygon():
    global current_mission, polygon_markers, polygon_path
    global home_marker, home_line, mission_path, waypoint_status

    for m in polygon_markers:
        m.delete()
    polygon_markers.clear()

    if polygon_path is not None:
        try: polygon_path.delete()
        except Exception as e: log_message(f"Error borrando polígono: {e}")
        polygon_path = None

    if home_marker: home_marker.delete()
    if home_line: home_line.delete()
    home_marker = None
    home_line = None

    if mission_path:
        mission_path.delete()
        mission_path = None

    current_mission = MissionData()
    waypoint_status = []

    home_position_var.set("")
    progress_var.set("Waypoints: 0/0")
    progress_bar['value'] = 0
    altitude_var.set(30)
    spacing_var.set(10)

    log_message("🔄 Reset completo: polígono, HOME y misión eliminados")

# =========================================
# [10] TRIMS / GRIPPER (COMANDOS Y UI)
# =========================================
def request_trim_data():
    """Solicita los trims al dron y deshabilita sliders mientras tanto."""
    for s in [slider_accel, slider_roll_lr, slider_roll_fb, slider_rudder, slider_switch]:
        s.config(state="disabled")
    send_command("GET_TRIMS", {})
    log_message("📡 Solicitando trims al dron...")
    print("[TX → LoRa/request_trim_data] Enviando comando GET_TRIMS")

def handle_trim_response(data):
    """Actualiza los sliders con la respuesta TRIM_DATA."""
    print(f"[RX ← LoRa/handle_trim_response] Datos recibidos TRIM_DATA → {data}")
    try:
        trim_accel_var.set(data.get("accel", 128))
        trim_roll_lr_var.set(data.get("roll_lr", 128))
        trim_roll_fb_var.set(data.get("roll_fb", 128))
        trim_rudder_var.set(data.get("rudder", 128))
        trim_switch_var.set(data.get("switch", 128))
        for slider in [slider_accel, slider_roll_lr, slider_roll_fb, slider_rudder, slider_switch]:
            slider.config(state="normal")
        log_message("✅ Trims actualizados y controles habilitados")
    except Exception as e:
        log_message(f"❌ Error procesando trims: {e}")

# =========================================
# [11] ARCHIVOS DE MISIÓN (GUARDAR/CARGAR)
# =========================================
def save_mission():
    if not current_mission.polygon_points:
        messagebox.showwarning("Error", "No hay misión para guardar")
        return
    filename = filedialog.asksaveasfilename(
        defaultextension=".mission",
        filetypes=[("Mission files", "*.mission"), ("All files", "*.*")]
    )
    if filename:
        try:
            with open(filename, 'wb') as f:
                pickle.dump(current_mission.to_dict(), f)
            log_message(f"Misión guardada: {filename}")
        except Exception as e:
            log_message(f"Error guardando misión: {str(e)}")

def load_mission():
    filename = filedialog.askopenfilename(
        filetypes=[("Mission files", "*.mission"), ("All files", "*.*")]
    )
    if filename:
        try:
            with open(filename, 'rb') as f:
                mission_data = pickle.load(f)
            reset_polygon()
            current_mission.from_dict(mission_data)
            update_interface_from_mission()
            log_message(f"Misión cargada: {filename}")
        except Exception as e:
            log_message(f"Error cargando misión: {str(e)}")

def update_interface_from_mission():
    for i, (lat, lon) in enumerate(current_mission.polygon_points):
        marker = mapa.set_marker(lat, lon, text=f"P{i+1}")
        polygon_markers.append(marker)
    if current_mission.home_position:
        set_home_position(current_mission.home_position[0], current_mission.home_position[1])
    altitude_var.set(current_mission.altitude)
    spacing_var.set(current_mission.spacing)
    takeoff_altitude_var.set(current_mission.takeoff_altitude)
    update_polygon_drawing()

# =========================================
# [12] CONSTRUCCIÓN DE LA GUI (TKINTER)
# =========================================
def create_trim_controls(frame):
    """Crea controles de TRIM + GRIPPER en pestaña Controles."""
    global trim_accel_var, trim_roll_lr_var, trim_roll_fb_var, trim_rudder_var, trim_switch_var
    global slider_accel, slider_roll_lr, slider_roll_fb, slider_rudder, slider_switch
    global gripper_state_var, handle_trim_response_global

    # Variables
    trim_accel_var = tk.IntVar(value=128)
    trim_roll_lr_var = tk.IntVar(value=128)
    trim_roll_fb_var = tk.IntVar(value=128)
    trim_rudder_var = tk.IntVar(value=128)
    trim_switch_var = tk.IntVar(value=128)
    gripper_state_var = tk.BooleanVar(value=False)

    main_frame = tk.LabelFrame(frame, text="Controles de Trim y Gripper", padx=15, pady=15, font=("Arial", 10, "bold"))
    main_frame.pack(fill="x", pady=10, padx=10)

    # ---- Columna Izquierda: Sliders ----
    trim_frame = tk.Frame(main_frame)
    trim_frame.grid(row=0, column=0, sticky="nsew", padx=(0, 20))

    sliders_info = [
        ("Acelerador", trim_accel_var),
        ("Rolido Izq/Der", trim_roll_lr_var),
        ("Rolido Del/Atr", trim_roll_fb_var),
        ("Timón", trim_rudder_var),
        ("Switch Control", trim_switch_var)
    ]
    slider_widgets = []
    for i, (label_text, var) in enumerate(sliders_info):
        tk.Label(trim_frame, text=label_text, font=("Arial", 9)).grid(row=i, column=0, sticky="w", pady=4)
        slider = tk.Scale(trim_frame, from_=0, to=255, orient="horizontal", variable=var, state="disabled", length=250)
        slider.grid(row=i, column=1, sticky="we", pady=4)
        slider.bind("<ButtonRelease-1>", lambda e, v=var: send_trim_data())
        slider_widgets.append(slider)
    slider_accel, slider_roll_lr, slider_roll_fb, slider_rudder, slider_switch = slider_widgets

    tk.Button(trim_frame, text="📡 Refrescar Datos", command=request_trim_data, bg="#2196F3", fg="white",
              font=("Arial", 9, "bold"), width=20).grid(row=len(sliders_info), column=0, columnspan=2, pady=10)

    # Envío de trims
    def send_trim_data(event=None):
        data = {
            "accel": trim_accel_var.get(),
            "roll_lr": trim_roll_lr_var.get(),
            "roll_fb": trim_roll_fb_var.get(),
            "rudder": trim_rudder_var.get(),
            "switch": trim_switch_var.get()
        }
        send_command("TRIM", data)
        log_message(f"Trim enviado: {data}")
        print(f"[TX → LoRa/send_trim_data] Enviando TRIM → {data}")

    # Hacer accesible el handler de TRIM_DATA para process_received_message
    handle_trim_response_global = handle_trim_response

    # ---- Columna Derecha: Gripper ----
    gripper_frame = tk.LabelFrame(main_frame, text="Gripper", padx=10, pady=10, font=("Arial", 10, "bold"))
    gripper_frame.grid(row=0, column=1, sticky="nsew")

    def toggle_gripper():
        state = gripper_state_var.get()
        send_command("GRIPPER", {"state": "OPEN" if state else "CLOSE"})
        print(f"[TX → LoRa/toggle_gripper] Enviando GRIPPER → {command}")
        if state:
            gripper_check.config(bg="#4CAF50", fg="white", text="ABIERTO")
            log_message("🟢 Gripper ABIERTO")
        else:
            gripper_check.config(bg="#f44336", fg="white", text="CERRADO")
            log_message("🔴 Gripper CERRADO")

    gripper_check = tk.Checkbutton(gripper_frame, variable=gripper_state_var,
                                   command=toggle_gripper, font=("Arial", 12, "bold"),
                                   indicatoron=False, width=15, bg="#f44336", fg="white",
                                   selectcolor="#4CAF50")
    gripper_check.pack(pady=20)

    # Estado inicial del botón
    if gripper_state_var.get():
        gripper_check.config(text="ABIERTO", bg="#4CAF50")
    else:
        gripper_check.config(text="CERRADO", bg="#f44336")

def create_gui():
    """Construye toda la interfaz (pestañas, botones, mapa, etc.)."""
    global root, mapa, status_label, telemetry_text, log_text
    global port_var, spacing_var, altitude_var, takeoff_altitude_var, status_message
    global home_position_var, progress_var, progress_bar
    global add_btn, home_btn, delete_btn
    global btn_arm, btn_disarm, btn_mission, btn_return
    global event_action_var

    root = tk.Tk()
    root.title("Ground Station - Dron Buscador Térmico")
    root.geometry("1200x900")

    event_action_var = tk.StringVar(value="VOLVER")

    # ---- Barra de conexión ----
    connection_frame = tk.Frame(root)
    connection_frame.pack(fill="x", padx=10, pady=5)

    ports = [port.device for port in serial.tools.list_ports.comports()]
    port_var = tk.StringVar(value=ports[0] if ports else "")

    tk.Label(connection_frame, text="Puerto:").pack(side="left")
    port_combo = ttk.Combobox(connection_frame, textvariable=port_var, values=ports, width=15)
    port_combo.pack(side="left", padx=5)

    def update_ports():
        new_ports = [port.device for port in serial.tools.list_ports.comports()]
        port_combo['values'] = new_ports
        if new_ports and not port_var.get():
            port_var.set(new_ports[0])

    tk.Button(connection_frame, text="Actualizar", command=update_ports).pack(side="left", padx=5)
    tk.Button(connection_frame, text="Conectar", command=connect_serial).pack(side="left", padx=5)
    tk.Button(connection_frame, text="Desconectar", command=disconnect_serial).pack(side="left", padx=5)

    status_label = tk.Label(connection_frame, text="Desconectado", fg="red")
    status_label.pack(side="right")

    # ---- Modo edición ----
    edit_frame = tk.Frame(root)
    edit_frame.pack(fill="x", padx=10, pady=5)

    tk.Label(edit_frame, text="Modo Edición:", font=("Arial", 10, "bold")).pack(side="left")
    add_btn = tk.Button(edit_frame, text="Colocar marcadores", command=lambda: set_edit_mode('polygon'), width=20)
    add_btn.pack(side="left", padx=2)
    home_btn = tk.Button(edit_frame, text="Colocar home", command=set_home_from_telemetry, width=20)
    home_btn.pack(side="left", padx=2)
    delete_btn = tk.Button(edit_frame, text="Eliminar marcadores", command=lambda: set_edit_mode('delete'), width=20)
    delete_btn.pack(side="left", padx=2)

    # ---- Parámetros de misión ----
    params_frame = tk.Frame(root)
    params_frame.pack(fill="x", padx=10, pady=5)

    tk.Label(params_frame, text="Altitud (m):").pack(side="left")
    altitude_var = tk.IntVar(value=30)
    tk.Spinbox(params_frame, from_=10, to=100, width=5, textvariable=altitude_var).pack(side="left", padx=5)

    tk.Label(params_frame, text="Ancho pasadas (m):").pack(side="left", padx=(20, 5))
    spacing_var = tk.IntVar(value=10)
    tk.Spinbox(params_frame, from_=5, to=50, width=5, textvariable=spacing_var).pack(side="left", padx=5)

    tk.Label(params_frame, text="Altura despegue (m):").pack(side="left", padx=(20, 5))
    takeoff_altitude_var = tk.IntVar(value=10)
    tk.Spinbox(params_frame, from_=5, to=100, width=5, textvariable=takeoff_altitude_var).pack(side="left", padx=5)

    home_position_var = tk.StringVar(value="")
    tk.Label(params_frame, textvariable=home_position_var, fg="green").pack(side="right")

    # ---- Estado y progreso ----
    status_frame = tk.Frame(root)
    status_frame.pack(fill="x", padx=10, pady=5)

    status_message = tk.StringVar(value="Modo: Polígono")
    tk.Label(status_frame, textvariable=status_message, fg="blue").pack(side="left")

    progress_var = tk.StringVar(value="Waypoints: 0/0")
    tk.Label(status_frame, textvariable=progress_var).pack(side="left", padx=20)

    progress_bar = ttk.Progressbar(status_frame, orient="horizontal", length=200, mode="determinate")
    progress_bar.pack(side="left", padx=5)

    # ---- Botones de misión ----
    mission_btn_frame = tk.Frame(root)
    mission_btn_frame.pack(fill="x", padx=10, pady=5)

    btn_mission = tk.Button(mission_btn_frame, text="COMENZAR MISIÓN", command=start_mission,
                            state="disabled", bg="lightgrey", fg="white")
    btn_mission.pack(side="left", expand=True, padx=5)

    btn_return = tk.Button(mission_btn_frame, text="REGRESAR", command=return_home,
                           state="disabled", bg="lightgrey", fg="white")
    btn_return.pack(side="left", expand=True, padx=5)

    tk.Button(mission_btn_frame, text="RESET COMPLETO", command=reset_polygon, bg="red", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="GUARDAR", command=save_mission, bg="blue", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="CARGAR", command=load_mission, bg="purple", fg="white").pack(side="left", expand=True, padx=5)

    tk.Label(mission_btn_frame, text="Acción tras evento:").pack(side="left", padx=5)
    event_action_menu = ttk.Combobox(mission_btn_frame, textvariable=event_action_var,
                                     values=["CONTINUAR", "VOLVER"], state="readonly", width=11)
    event_action_menu.pack(side="left", padx=5)
    event_action_menu.current(1)  # "VOLVER"

    # ---- Notebook ----
    notebook = ttk.Notebook(root)
    notebook.pack(fill="both", expand=True, padx=10, pady=5)

    # Pestaña: Misión
    frame_mission = ttk.Frame(notebook)
    notebook.add(frame_mission, text="Misión")

    # Mapa
    global mapa
    mapa = tkintermapview.TkinterMapView(frame_mission, width=1000, height=500, corner_radius=0)
    mapa.pack(pady=10, fill="both", expand=True)
    mapa.set_position(-34.6037, -58.3816)
    mapa.set_zoom(12)
    mapa.add_left_click_map_command(map_click)

    # Pestaña: Controles
    frame_controls = ttk.Frame(notebook)
    notebook.add(frame_controls, text="Controles")

    control_btn_frame = tk.Frame(frame_controls)
    control_btn_frame.pack(pady=10)

    global btn_arm, btn_disarm
    btn_arm = tk.Button(control_btn_frame, text="ARMAR", command=arm_drone, bg="green", fg="white", width=15)
    btn_arm.pack(side="left", padx=10)

    btn_disarm = tk.Button(control_btn_frame, text="DESARMAR", command=disarm_drone,
                           state="disabled", bg="lightgrey", fg="white", width=15)
    btn_disarm.pack(side="left", padx=10)

    create_trim_controls(frame_controls)

    # Pestaña: Telemetría
    frame_telemetry = ttk.Frame(notebook)
    notebook.add(frame_telemetry, text="Telemetría")

    global telemetry_text
    telemetry_text = scrolledtext.ScrolledText(frame_telemetry, width=80, height=15)
    telemetry_text.pack(pady=10, fill="both", expand=True)
    telemetry_text.insert("end", "Esperando datos de telemetría...\n")
    telemetry_text.config(state="disabled")

    # Pestaña: Log
    frame_log = ttk.Frame(notebook)
    notebook.add(frame_log, text="Log")

    global log_text
    log_text = scrolledtext.ScrolledText(frame_log, width=80, height=15)
    log_text.pack(pady=10, fill="both", expand=True)
    log_text.config(state="disabled")

    # Modo inicial
    set_edit_mode('polygon')

# =========================
# [13] INICIALIZACIÓN
# =========================
current_mission = MissionData()

# Ejecutar GUI 
create_gui()
log_message("✅ Interfaz lista. Conectá el puerto y espera telemetría...")
root.mainloop()



[SERIAL] ✅ Conectado a COM1 @ 115200 bps
[SERIAL] 🚀 Lanzando hilos: recepción y ack_manager
[SERIAL] ✅ Puerto cerrado correctamente
